<div style ="font-family:Trebuchet MS; background-color : #f8f0fa; border-left: 5px solid #1b4332; padding: 12px; border-radius: 50px 50px;">
    <h2 style="color: #1b4332; font-size: 48px; text-align: center;"><b> Dog-Cat Classification
 </b></h2>

In [ ]:
import numpy as np
import pandas as pd
import os
import gc
import matplotlib.pyplot as plt
import seaborn as sns
import random
import matplotlib.image as mpimg
import cv2
import zipfile
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Activation, BatchNormalization,Dense, Dropout,GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Images Exploration & Analysis

In [ ]:
zip_file_path = '/kaggle/input/dogs-vs-cats/train.zip'
extract_folder = '/kaggle/working/pictures'


os.makedirs(extract_folder, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_folder)
print(os.listdir(extract_folder))


In [ ]:
from PIL import Image

# Folder where the images are extracted
image_folder = '/kaggle/working/pictures/train'

# Get a list of image filenames
image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Show sample images with their filenames
num_samples = min(5, len(image_files))  # Display up to 5 images

plt.figure(figsize=(10, 10))
for i, image_file in enumerate(image_files[:num_samples]):
    img_path = os.path.join(image_folder, image_file)
    img = Image.open(img_path)
    
    # Display the image
    plt.subplot(1, num_samples, i + 1)
    plt.imshow(img)
    plt.title(image_file, fontsize=8)
    plt.axis('off')

plt.show()


In [ ]:
len(image_files)

In [ ]:
print(image_files[4])

In [ ]:
image_files[0].split('.')[0]

In [ ]:
cats = 0
dogs = 0
for file in image_files :
    if file.split('.')[0] == 'dog' :
        dogs += 1
    else :
        cats +=1 

cats,dogs
    

In [ ]:
# Labels and data for the pie chart
labels = ['Dogs', 'Cats']
sizes = [dogs, cats]
colors = ['#ff9999','#66b3ff']  
# Create a pie chart
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors)
plt.axis('equal')
plt.title('Distribution of Dogs and Cats')
plt.show()

**we have a balanced data**

In [ ]:
del image_files
gc.collect()

# Data Preprocessing

In [ ]:
IMAGE_SIZE = (224,224)
def load_data(image_dir_train):
    def process_images(image_dir):
        images = []
        print("Loading images from {}".format(image_dir))
        for image_file in tqdm(os.listdir(image_dir)):
            img_path = os.path.join(image_dir, image_file)

            # Open and resize the image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, IMAGE_SIZE)

            images.append(image)

        return np.array(images, dtype='float32')

    # Process train images and labels
    train_images = process_images(image_dir_train)
    labels = []
    for image_file in tqdm(os.listdir(image_dir_train)):
        image_class = image_file.split('.')[0]
        if image_class == 'dog' :
            labels.append(1)
        else :
            labels.append(0)
    train_labels = np.array(labels, dtype='int32')
    return train_images,train_labels 

train_images , train_labels = load_data(image_folder)
 

In [ ]:
mymap = {
    0 : 'cat' ,
    1 : 'dog'
}

In [ ]:
# Show the image (you must normalise it between 0 to 1)
def show_random(samples=5) :
    plt.figure(figsize=(10, 10))
    for i in range(samples) :
        r = random.randint(0, 25000)
        plt.subplot(1, samples, i + 1)
        plt.imshow(train_images[r] / 255.0)
        plt.title(f"Label: {mymap[train_labels[r]]}")
        plt.axis('off')
    plt.tight_layout()  # Adjust spacing to make sure images fit well
    plt.show()
show_random()

# CNN creation 

In [ ]:
from sklearn.model_selection import train_test_split
train_images = train_images.astype('uint8')
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(train_images, train_labels, test_size=0.2,shuffle=True, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Validation data shape: {X_val.shape}")

In [ ]:
from collections import Counter

print("Train labels distribution:", Counter(y_train))
print("Validation labels distribution:", Counter(y_val))


In [ ]:
del train_images,train_labels
gc.collect()

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3),input_shape = (224,224,3) , activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3),  activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.4),  # Dropout to prevent overfitting
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

**adding callback early stop**

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [ ]:
BATCH_SIZE = 32
# datagen3 = ImageDataGenerator(
#     rescale=1.0/255.0,
# )
# train_generator3 = datagen3.flow(X_train, y_train, BATCH_SIZE,shuffle=True)
val_datagen = ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow(X_val, y_val, BATCH_SIZE)

In [ ]:
def train_model_with_generator(train_gen, batch_size=32, epochs=15):
    history = model.fit(
    train_gen,
    validation_data= val_generator,
    epochs=epochs,
    callbacks=[early_stopping],
        )

    
    return history

In [ ]:
def plot_loss(history) :
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")


In [ ]:
def plot_acc(history) :
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.grid(True)
    plt.show()

    # Evaluate the model
    val_loss, val_accuracy = model.evaluate(X_val, y_val, verbose=0)
    print(f"Validation Loss: {val_loss:.4f}")
    print(f"Validation Accuracy: {val_accuracy:.4f}")

In [ ]:
# history =  train_model_with_generator(train_generator3)

In [ ]:
# plot_acc(history)
# plot_loss(history)

In [ ]:
# del train_generator3,datagen3,history
# import gc
# gc.collect()


# Data Augmentation

In [ ]:
datagen2 = ImageDataGenerator(
    rescale=1.0/255.0,
    horizontal_flip=True,     # Randomly flip half the images horizontally
    zoom_range=[0.9, 1.1],    # Slightly conservative zoom
    brightness_range=[0.8, 1.2], # Randomly change brightness
    fill_mode='nearest'       # Fill in new pixels using reflection to avoid artifacts
)
train_generator2 = datagen2.flow(X_train, y_train, BATCH_SIZE)

In [ ]:
history_image_augmentation = train_model_with_generator(train_generator2)

In [ ]:
plot_acc(history_image_augmentation)
plot_loss(history_image_augmentation)

**we got better results with data augmentation**

# Make Prediction & Save the Model

In [ ]:
# model.save('rafik_dog_cat_class.h5')

In [ ]:
from tensorflow.keras.models import load_model

model = load_model('/kaggle/input/rafik_cat_dog/keras/default/1/rafik_dog_cat_class.h5')


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

mymap = {
    0 : 'cat' ,
    1 : 'dog'
}
model = load_model('/kaggle/input/rafik_cat_dog/keras/default/1/rafik_dog_cat_class.h5')


def preprocess_image(image_path, target_size=(224, 224)):
    img = image.load_img(image_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) 
    img_array = img_array / 255.0  
    return img_array

img_array = preprocess_image('/kaggle/input/cat-dog-test/dog3.jpg')
predictions = model.predict(img_array)
print(predictions)
predicted_class = (predictions > 0.5).astype("int32")
print(f"Predicted class: {predicted_class}")
plt.imshow(np.squeeze(img_array, axis=0))
plt.title(f"Label: {mymap[predicted_class[0][0]]}")
plt.axis('off')
plt.tight_layout()  
plt.show()


# Submit result 

In [ ]:
# sample_submission = pd.read_csv('/kaggle/input/dogs-vs-cats/sampleSubmission.csv')
# sample_submission

In [ ]:
# zip_file_path = '/kaggle/input/dogs-vs-cats/test1.zip'
# extract_folder = '/kaggle/working/pictures'


# os.makedirs(extract_folder, exist_ok=True)

# # Extract the zip file
# with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#     zip_ref.extractall(extract_folder)
# print(os.listdir(extract_folder))


In [ ]:
# from PIL import Image

# # Folder where the images are extracted
# image_folder = '/kaggle/working/pictures/test1'

# image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]
# num_samples = min(5, len(image_files))

# plt.figure(figsize=(10, 10))
# for i, image_file in enumerate(image_files[:num_samples]):
#     img_path = os.path.join(image_folder, image_file)
#     img = Image.open(img_path)
    
#     # Display the image
#     plt.subplot(1, num_samples, i + 1)
#     plt.imshow(img)
#     plt.title(image_file, fontsize=8)
#     plt.axis('off')

# plt.show()


In [ ]:
# IMAGE_SIZE = (224,224)
# def load_data(image_dir):
#     """
#     Load and preprocess the test images.
#     """
#     images = []
#     ids = []

#     print("Loading test images from {}".format(image_dir))
#     for image_file in tqdm(os.listdir(image_dir)):
#         img_path = os.path.join(image_dir, image_file)
        
#         # Load and preprocess the image
#         img = cv2.imread(img_path)
#         img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#         img = cv2.resize(img, IMAGE_SIZE)

       
#         img = img / 255.0
        
#         images.append(img)
#         img_id = image_file.split('.')[0]  # Extract the image ID (without extension)
#         ids.append(img_id)

#     return np.array(images, dtype='float32'), ids

# # Load test images
# test_images, ids = load_data('/kaggle/working/pictures/test1')

# print("Making predictions on the test images...")
# predictions = model.predict(test_images)

# # Convert predictions to labels
# predicted_labels = (predictions > 0.5).astype('int32').flatten()


# submission_df = pd.DataFrame({
#     'id': ids,
#     'label': predicted_labels
# })
# submission_csv_path = 'submission.csv' 
# submission_df = submission_df.sort_values(by='id')
# submission_df.to_csv(submission_csv_path, index=False)
# print(f"Submission file saved: {submission_csv_path}")

In [ ]:
# output_submission = pd.read_csv('/kaggle/working/submission.csv')
# output_submission